In [15]:
import pandas as pd
from bs4 import BeautifulSoup
import json

# df = pd.read_parquet("../part-00000-8bcd0748-094a-4de0-8574-2328d1833c16-c000.gz.parquet", engine='pyarrow')
# df = pd.read_parquet("../part-00000-ae0248c1-9d9a-4943-8350-71a5267f9c51-c000.gz.parquet", engine='pyarrow')
df = pd.read_parquet("../part-00000-96ac3cdd-9fe4-4189-a066-f7b73d199c16-c000.gz.parquet", engine='pyarrow')
#df = pd.read_parquet("../part-00000-8ef97ec5-2cb7-4b7a-8a07-0935e967ee8e-c000.gz.parquet", engine='pyarrow')
df
# print("done")

,key,surtUrl,timestamp,originalUrl,mime,status,digest,redirectUrl,meta,contentLength,offset,filename,allheader,payload
0,"http://(com,visitcalifornia,www,)/ca/destinati...","http://(com,visitcalifornia,www,)/ca/destinati...",20170215,http://www.visitcalifornia.com/ca/destination/...,text/html,200,-,-,-,151417,364,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,"HTTP/1.1 200 OK\r\nDate: Wed, 15 Feb 2017 12:4..."
1,"http://(com,visitcalifornia,www,)/ca/destinati...","http://(com,visitcalifornia,www,)/ca/destinati...",20170412,http://www.visitcalifornia.com/ca/destination/...,text/html,200,-,-,-,152979,23226,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,"HTTP/1.1 200 OK\r\nDate: Wed, 12 Apr 2017 02:1..."
2,"http://(com,visitcalifornia,www,)/ca/destinati...","http://(com,visitcalifornia,www,)/ca/destinati...",20170606,http://www.visitcalifornia.com/ca/destination/...,text/html,200,-,-,-,155277,46421,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,"HTTP/1.1 200 OK\r\nDate: Tue, 06 Jun 2017 10:2..."
3,"http://(com,visitcalifornia,www,)/ca/destinati...","http://(com,visitcalifornia,www,)/ca/destinati...",20171220,http://www.visitcalifornia.com/ca/destination/...,text/html,200,-,-,-,164159,69799,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,"HTTP/1.1 200 OK\r\nDate: Wed, 20 Dec 2017 06:4..."
4,"https://(com,visitcalifornia,www,)/ca/destinat...","https://(com,visitcalifornia,www,)/ca/destinat...",20181223,https://www.visitcalifornia.com/ca/destination...,text/html,200,-,-,-,182514,96185,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,"HTTP/1.1 200 OK\r\nDate: Sun, 23 Dec 2018 03:0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58988,"http://(com,visitcalifornia,www,)/in/node/2689...","http://(com,visitcalifornia,www,)/in/node/26891",20170618,http://www.visitcalifornia.com/in/node/26891,text/html,200,-,-,-,222605,1073583698,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,"HTTP/1.1 200 OK\r\nDate: Sun, 18 Jun 2017 05:3..."
58989,"http://(com,visitcalifornia,www,)/in/node/2689...","http://(com,visitcalifornia,www,)/in/node/26891",20171221,http://www.visitcalifornia.com/in/node/26891,text/html,200,-,-,-,253014,1073615343,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,"HTTP/1.1 200 OK\r\nDate: Thu, 21 Dec 2017 17:0..."
58990,"https://(com,visitcalifornia,www,)/in/node/268...","https://(com,visitcalifornia,www,)/in/node/26891",20181226,https://www.visitcalifornia.com/in/node/26891,text/html,200,-,-,-,282648,1073653279,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,"HTTP/1.1 200 OK\r\nDate: Wed, 26 Dec 2018 13:5..."
58991,"http://(com,visitcalifornia,www,)/in/node/2689...","http://(com,visitcalifornia,www,)/in/node/2689...",20161225,http://www.visitcalifornia.com/in/node/26891/i...,text/html,200,-,-,-,197041,1073697150,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,"HTTP/1.1 200 OK\r\nDate: Sun, 25 Dec 2016 03:4..."


In [16]:
df.timestamp[0]

'20170215'

In [17]:
# df['timestamp'] = pd.to_datetime(df['timestamp'], infer_datetime_format=True)

In [18]:
def headerAndTime(df):
    #list that will hold information for every row 
    tableInfo = []
    #list that will hold indices of rows with errors
    errors = []
    for row in range(df.shape[0]):
        try:
            #dictionary to store information from the row
            quickDict = {}
            #grabbing information from each column of the row
            quickDict["Row"] = row
            soup = BeautifulSoup(df.payload[row], 'html.parser')
            quickDict["Header"] = soup.title.text
            quickDict["Timestamp"] = df.timestamp[row]
            quickDict["URL"] = df.originalUrl[row]
            tableInfo.append(quickDict)
        except AttributeError:
            #some rows don't have an HTML file in the payload
            #when trying to use soup.title.text above, it will give an AttributeError because it has no title.text attribute
            #this exception adds the row index to a list titled "errors" to be able to see which rows had the error in case it's needed
            errors.append(row)
    return tableInfo, errors

In [19]:
tableInfo, errors = headerAndTime(df)

In [20]:
print(len(errors))

12429


In [21]:
import csv
csv_columns = ['Row','Header','Timestamp', 'URL']
csv_file = "headerTimestampUrl3.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in tableInfo:
            writer.writerow(data)
except IOError:
    print("I/O error")